# Clean data

In this notebook, we are cleaning the data and converting it into AlkgieV1 Entities.

This step is important in generating ids and further processing the data into a coherent set of domain specific entities that the Alkgie project can use.

## Flatten the data

In [1]:
#r "nuget:FSharp.Data"

open FSharp.Data
type ScrappedData = JsonProvider<"../data/scrapped/scrapped-dataset.json">
let datasets = ScrappedData.Load("../data/scrapped/scrapped-dataset.json")

type FlattenedData = {
    Link: Option<string>
    Name: Option<string>
    Description: string
    Headers: string[]
    AwesomeList: string
}

let flattenedData = 
    datasets
    |> Seq.collect (fun dataset ->
        dataset.Data
        |> Seq.map (fun item -> 
            {
                Link = item.Link
                Name = item.Name
                Description = item.Description
                Headers = item.Headers
                AwesomeList = dataset.Filename
            }
        )
    )
    |> Seq.toList
flattenedData


Installed Packages FSharp.Data, 6.4.0

[ { Link = Some "#"
 Name =
 Some
 "Awesome F#F# wrappers for popular .NET librariesActor frameworksBuild toolsCloudCompilersConcurrent, asynchronous and parallel programmingConfigurationData ScienceDevelopment ToolsIDEEditor pluginsPerformance analysisGame developmentGeneral purpose librariesGUIHTTP ClientsLoggingPackage ManagementParsingPreCompilationSearchSerializationSimulationTestingType providersCreating type providersVisualizationWeb frameworks.Net Core TemplatesResourcesBlogsCheatsheetsCommunityOther ListsWebsitesVideosCourses"
 Description =
 "Awesome F#
F# wrappers for popular .NET libraries
Actor frameworks
Build tools
Cloud
Compilers
Concurrent, asynchronous and parallel programming
Configuration
Data Science
Development Tools
IDE
Editor plugins
Performance analysis


Game development
General purpose libraries
GUI
HTTP Clients
Logging
Package Management
Parsing
PreCompilation
Search
Serialization
Simulation
Testing
Type providers
Creating type providers


Visualization
Web frameworks
.Net Core Templates
Resources
Blogs
Cheatsheets
Community
Other Lists
Websites
Videos
Courses



"
 Headers = [|"Table of Contents"; " Awesome F#"|]
 AwesomeList = "awesome-fsharp.md" }, { Link = Some "#f-wrappers-for-popular-net-libraries"
 Name = Some "F# wrappers for popular .NET libraries"
 Description = "F# wrappers for popular .NET libraries"
 Headers = [|"Table of Contents"; " Awesome F#"|]
 AwesomeList = "awesome-fsharp.md" }, { Link = Some "#actor-frameworks"
 Name = Some "Actor frameworks"
 Description = "Actor frameworks"
 Headers = [|"Table of Contents"; " Awesome F#"|]
 AwesomeList = "awesome-fsharp.md" }, { Link = Some "#build-tools"
 Name = Some "Build tools"
 Description = "Build tools"
 Headers = [|"Table of Contents"; " Awesome F#"|]
 AwesomeList = "awesome-fsharp.md" }, { Link = Some "#cloud"
 Name = Some "Cloud"
 Description = "Cloud"
 Headers = [|"Table of Contents"; " Awesome F#"|]
 AwesomeList = "awesome-fsharp.md" }, { Link = Some "#compilers"
 Name = Some "Compilers"
 Description = "Compilers"
 Headers = [|"Table of Contents"; " Awesome F#"|]
 AwesomeList = "awesome-fsharp.md" }, { Link = Some "#concurrent-asynchronous-and-parallel-programming"
 Name = Some "Concurrent, asynchronous and parallel programming"
 Description = "Concurrent, asynchronous and parallel programming"
 Headers = [|"Table of Contents"; " Awesome F#"|]
 AwesomeList = "awesome-fsharp.md" }, { Link = Some "#configuration"
 Name = Some "Configuration"
 Description = "Configuration"
 Headers = [|"Table of Contents"; " Awesome F#"|]
 AwesomeList = "awesome-fsharp.md" }, { Link = Some "#data-science"
 Name = Some "Data Science"
 Description = "Data Science"
 Headers = [|"Table of Contents"; " Awesome F#"|]
 AwesomeList = "awesome-fsharp.md" }, { Link = Some "#development-tools"
 Name = Some "Development ToolsIDEEditor pluginsPerformance analysis"
 Description = "Development Tools
IDE
Editor plugins
Performance analysis

"
 Headers = [|"Table of Contents"; " Awesome F#"|]
 AwesomeList = "awesome-fsharp.md" }, { Link = Some "#ide"
 Name = Some "IDE"
 Description = "IDE"
 Headers = [|"Table of Contents"; " Awesome F#"|]
 AwesomeList = "awesome-fsharp.md" }, { Link = Some "#editor-plugins"
 Name = Some "Editor plugins"
 Description = "Editor plugins"
 Headers = [|"Table of Contents"; " Awesome F#"|]
 AwesomeList = "awesome-fsharp.md" }, { Link = Some "#performance-analysis"
 Name = Some "Performance analysis"
 Description = "Performance analysis"
 Headers = [|"Table of Contents"; " Awesome F#"|]
 AwesomeList = "awesome-fsharp.md" }, { Link = Some "#game-development"
 Name = Some "Game development"
 Description = "Game development"
 Headers = [|"Table of Contents"; " Awesome F#"|]
 AwesomeList = "awesome-fsharp.md" }, { Link = Some "#general-purpose-libraries"
 Name = Some "General purpose libraries"
 Description = "General purpose libraries"
 Headers = [|"Table of Contents"; " Awesome F#"|]
 AwesomeList = "awesome-fsharp.md" }, { Link = Some "#gui"
 

## Transform flattened data into AlkgieV1 entities.

AlkgieV1 entities is the name I'm giving to the data format / schema that this project produces. It's not a final format, hence the V1.

Borrowing inspiration from thematic analysis, these entities define their names as "codes", as in an identified code in thematic analysis.

Themes are groupings of codes that form a cohesive whole.

In most instances I expect codes to be specific software products such as programming lanugages (like f#, javascript), whereas I expect themes to be more related to concepts (such as the concept of programming languages itself).

We are also assigning an unique Id at this stage to make it easier to graph things. These Ids are non-stable between versions of the dataset. Hopefully a future version of this project will find a way to keep ids stable between versions.

So essentially this is the Relationship Classification (RC) stage of this data analysis project.

In [2]:
type AlkgieV1Relation = {
    Id: Guid
    Source: string
}

type AlkgieV1EntityTypes =
    | Theme
    | Code

type AlkgieV1Entity = {
    Id: Guid
    Relations: AlkgieV1Relation[]
    Link: Option<string>
    Name: Option<string>
    Description: string
    Source: string
    EntityType: AlkgieV1EntityTypes
}

// Collect headers as Themes
let themePartials =
    flattenedData
    |> Seq.collect (fun item -> 
        // For each item, create a sequence of tuples (header, source)
        item.Headers |> Seq.map (fun header -> (header, item.AwesomeList))
    )
    |> Seq.distinct
    |> Seq.map (fun (header, source) -> 
        {|
            Id = Guid.NewGuid()
            Source = source
            Name = header
        |}
    )

// Helper functions
let getEntityType source name =
    match name with
    | None -> Code
    | Some actualName ->
        if themePartials |> Seq.exists (fun theme -> theme.Source = source && theme.Name = actualName) then
            Theme
        else
            Code

let getRelations source headers =
    themePartials
    |> Seq.filter (fun theme -> theme.Source = source && headers |> Seq.exists (fun header -> header = theme.Name))
    |> Seq.map (fun theme -> { Id = theme.Id; Source = theme.Source })

let getId source name =
    match name with
    | None -> Guid.NewGuid()
    | Some actualName ->
        match themePartials |> Seq.tryFind (fun theme -> theme.Source = source && theme.Name = actualName) with
        | Some(theme) -> theme.Id
        | None -> Guid.NewGuid()

 // Generate entities
let entities =
    flattenedData
    |> Seq.map (fun item -> 
        {
            Id = getId item.AwesomeList item.Name
            Relations = getRelations item.AwesomeList item.Headers |> Seq.toArray
            Link = item.Link
            Name = item.Name
            Description = item.Description
            Source = item.AwesomeList
            EntityType = getEntityType item.AwesomeList item.Name
        }
    )

entities

[ { Id = 68f34f0c-fdd4-497e-9c05-995667ebaf72
 Relations =
 [|{ Id = 61d6e3e1-2114-414b-95e5-164b04737e4a
 Source = "awesome-fsharp.md" };
 { Id = 592cf4fa-9168-4c50-bea9-fc100b481051
 Source = "awesome-fsharp.md" }|]
 Link = Some "#"
 Name =
 Some
 "Awesome F#F# wrappers for popular .NET librariesActor frameworksBuild toolsCloudCompilersConcurrent, asynchronous and parallel programmingConfigurationData ScienceDevelopment ToolsIDEEditor pluginsPerformance analysisGame developmentGeneral purpose librariesGUIHTTP ClientsLoggingPackage ManagementParsingPreCompilationSearchSerializationSimulationTestingType providersCreating type providersVisualizationWeb frameworks.Net Core TemplatesResourcesBlogsCheatsheetsCommunityOther ListsWebsitesVideosCourses"
 Description =
 "Awesome F#
F# wrappers for popular .NET libraries
Actor frameworks
Build tools
Cloud
Compilers
Concurrent, asynchronous and parallel programming
Configuration
Data Science
Development Tools
IDE
Editor plugins
Performance analysis


Game development
General purpose libraries
GUI
HTTP Clients
Logging
Package Management
Parsing
PreCompilation
Search
Serialization
Simulation
Testing
Type providers
Creating type providers


Visualization
Web frameworks
.Net Core Templates
Resources
Blogs
Cheatsheets
Community
Other Lists
Websites
Videos
Courses



"
 Source = "awesome-fsharp.md"
 EntityType = Code }, { Id = 00f4c1bf-874b-4939-a253-f3923b1439d0
 Relations =
 [|{ Id = b277a65b-8c60-4b85-81ec-7a628dda2545
 Source = "awesome-fsharp.md" };
 { Id = 4b376265-c2ba-464e-b2d6-099fa5fef63e
 Source = "awesome-fsharp.md" }|]
 Link = Some "#f-wrappers-for-popular-net-libraries"
 Name = Some "F# wrappers for popular .NET libraries"
 Description = "F# wrappers for popular .NET libraries"
 Source = "awesome-fsharp.md"
 EntityType = Code }, { Id = 7cca441a-6de3-4502-bc2e-2b2f555da180
 Relations =
 [|{ Id = 1bd04fc9-08d3-407c-8e64-2e468a54c492
 Source = "awesome-fsharp.md" };
 { Id = a595bd32-5e80-4e34-b820-6b432e835adf
 Source = "awesome-fsharp.md" }|]
 Link = Some "#actor-frameworks"
 Name = Some "Actor frameworks"
 Description = "Actor frameworks"
 Source = "awesome-fsharp.md"
 EntityType = Theme }, { Id = 391ecc21-39cc-4f0d-a681-a387ee813ea1
 Relations =
 [|{ Id = 1b30680c-59b7-4285-ad4b-621a775061f6
 Source = "awesome-fsharp.md" };
 { Id = cfc5da8d-0232-4403-92d4-e36b7ae49912
 Source = "awesome-fsharp.md" }|]
 Link = Some "#build-tools"
 Name = Some "Build tools"
 Description = "Build tools"
 Source = "awesome-fsharp.md"
 EntityType = Theme }, { Id = ef54f259-14fc-48a8-8eb1-8e92a5fb4ebf
 Relations =
 [|{ Id = 22688439-4723-4c5c-9e91-818eb03ddf0c
 Source = "awesome-fsharp.md" };
 { Id = c2c457d2-cec6-4104-8e12-2e3c616b5fa4
 Source = "awesome-fsharp.md" }|]
 Link = Some "#cloud"
 Name = Some "Cloud"
 Description = "Cloud"
 Source = "awesome-fsharp.md"
 EntityType = Theme }, { Id = 071cd045-51cf-4db1-acb0-2dd8799310ae
 Relations =
 [|{ Id = 2989f8c6-964a-4803-8145-386ffed79106
 Source = "awesome-fsharp.md" };
 { Id = 2fdda0ab-ce06-4753-8647-6ec847fb61ea
 Source = "awesome-fsharp.md" }|]
 Link = Some "#compilers"
 Name = Some "Compilers"
 Description = "Compilers"
 Source = "awesome-fsharp.md"
 EntityType = Theme }, { Id = f250c387-9331-4ba9-b5bf-bfff99a4467c
 Relations =
 [|{ Id = 389531b1-8846-40dd-b8d3-d7b12b5fa676
 Source = "awesome-fsharp.md" };
 { Id = cb08f910-060e-47f8-b2a5-49e7cb30d9a0
 Source = "awesome-fsharp.md" }|]
 Link = Some "#concurrent-asynchronous-and-parallel-programming"
 Name = Some "Concurrent, asynchronous and parallel programming"
 Description = "Concurrent, asynchronous and parallel programming"
 Source = "awesome-fsharp.md"
 EntityType = Theme }, { Id = 2536000d-800b-4d30-83dc-85c66bc360fc
 Relations =
 [|{ Id = 2addb6a0-5f3a-4e14-9435-5f9d27f971f2
 Source = "awesome-fsharp.md" };
 { Id = 2b61d844-abf1-49a6-a1d7-9a48557fdb97
 Source = "awesome-fsharp.md" }|]
 Link = Some "#configuration"
 Name = Some "Configuration"
 Description = "Configuration"
 Source = "awesom

## Save cleaned data

Saving the results of data cleaning

In [6]:

// This is a temporary hack to get around not being able to seralize Discriminated Unions
let entityTypeToString entityType =
    match entityType with
    | Theme -> "Theme"
    | Code -> "Code"

let temp =
    entities
    |> Seq.map( fun entity -> {|entity with EntityType = entityTypeToString entity.EntityType|})


// Actual saving
open System.IO
open System.Text.Json

let filePath = "../data/cleaned/cleaned-dataset.json"
let json = JsonSerializer.Serialize(temp, JsonSerializerOptions(WriteIndented = true))

let directoryPath = Path.GetDirectoryName(filePath)
if not <| Directory.Exists(directoryPath) then
    Directory.CreateDirectory(directoryPath) |> ignore

File.WriteAllText(filePath, json)